In [ ]:
import camb


lmax = 8150

cosmology_args = dict( # Junk value
    H0=63.3,
    mnu=0.531,
    ombh2=0.0217,
    omch2=0.115,
    tau=0.594
)
init_power_args = dict( # Junk value
    ns=0.947
)

pars = camb.CAMBparams()
pars.set_cosmology(**cosmology_args)
pars.InitPower.set_params(**init_power_args)
pars.set_for_lmax(lmax, lens_potential_accuracy=0)

results = camb.get_results(pars)

lmax_derived = results._lmax_setting()  # 8200

print(f"Input lmax:{lmax}; derived lmax: {lmax_derived}")

Input lmax:8150; derived lmax: 2400


# Details

In [1]:
import camb

In [2]:
lmax = 8150

The cosmological parameters are unimportant, they're simply values I had readily at hand.

In [ ]:
cosmology_args = dict( # Junk value
    H0=63.3,
    mnu=0.531,
    ombh2=0.0217,
    omch2=0.115,
    tau=0.594
)
init_power_args = dict( # Junk value
    ns=0.947
)

Set up CAMBparams in what I understand to be the typical manner.

In [ ]:
pars = camb.CAMBparams()
pars.set_cosmology(**cosmology_args)
pars.InitPower.set_params(**init_power_args)

When I use `set_for_lmax()`, the default setting `lens_margin=150`. From documentation [here](https://camb.readthedocs.io/en/latest/model.html#camb.model.CAMBparams.set_for_lmax):

> Set parameters to get CMB power spectra accurate to specific a `l_lmax`. Note this does not fix the actual output L range, spectra may be calculated above `l_max` (but may not be accurate there). To fix the l_max for output arrays use the optional input argument to `results.CAMBdata.get_cmb_power_spectra()` etc.

In [6]:
pars.set_for_lmax(lmax, lens_potential_accuracy=0)
print(pars.max_l)

8300


In [ ]:
results = camb.get_results(pars)

Get the lmax as determined by CAMB with `_lmax_setting()`. Internal to `save_cmb_power_spectrum()`, this method gets the maximum ell from some fortran function (unknown).

Documentation for `save_cmb_power_spectrum()`:
> Save CMB power to a plain text file. Output is lensed total then lensing potential and cross: L TT EE BB TE PP PT PE.

Documentation for `get_cmb_power_spectrum()`:
> Get CMB power spectra, as requested by the ‘spectra’ argument. All power spectra are self-owned numpy arrays (0..lmax, 0..3), where 0..3 index are TT, EE, BB, TE, unless raw_cl is True in which case return just . For the lens_potential the power spectrum returned is that of the deflection.
> 
> Note that even if lmax is None, all spectra a returned to the same lmax, appropriate for lensed spectra. Use the individual functions instead if you want to the full unlensed and lensing potential power spectra to the higher lmax actually computed.

In [4]:
lmax_derived = results._lmax_setting()  # 8200

In [5]:
print(f"Input lmax: {lmax}; lmax_derived: {lmax_derived}")

Input lmax: 8150; lmax_derived: 8200


It took some digging to figure out what's going on. I do not know how to look up the Fortran function being used.

My confusion arises because I expect that the `lmax` would be what I set it to be. I read the documentation for `save_cmb_power_spectrum()` but it's minimal. I checked `get_cmb_power_spectrum()`, which states that the lmax returned is appropriate for the lensed spectra. Only later did I check `set_for_lmax()`, where I was suprised that the `lmax` I set would be different from the output.

I think the simplest solution would be to update the documentation for `save_cmb_power_spectrum()` and `get_cmb_power_spectrum()`. It seems that use of `set_for_lmax()` is expected. 

Alternatively, further work could be done to ensure the value is used consistently.

## Original

Documentation for `set_for_lmax()`:
>        Set parameters to get CMB power spectra accurate to specific a l_lmax.
>        Note this does not fix the actual output L range, spectra may be calculated above l_max
>        (but may not be accurate there). To fix the l_max for output arrays use the optional input argument
>        to :meth:`.results.CAMBdata.get_cmb_power_spectra` etc.
>
>        :param lmax: :math:`\ell_{\rm max}` you want
>        :param max_eta_k: maximum value of :math:`k \eta_0\approx k\chi_*` to use, which indirectly sets k_max.
>                          If None, sensible value set automatically.
>        :param lens_potential_accuracy: Set to 1 or higher if you want to get the lensing potential accurate
>                                        (1 is only Planck-level accuracy)
>        :param lens_margin: the :math:`\Delta \ell_{\rm max}` to use to ensure lensed :math:`C_\ell` are correct
>                            at :math:`\ell_{\rm max}`
>        :param k_eta_fac:  k_eta_fac default factor for setting max_eta_k = k_eta_fac*lmax if max_eta_k=None
>        :param lens_k_eta_reference:  value of max_eta_k to use when lens_potential_accuracy>0; use
>                                      k_eta_max = lens_k_eta_reference*lens_potential_accuracy
>        :param nonlinear: use non-linear power spectrum; if None, sets nonlinear if lens_potential_accuracy>0 otherwise
>                          preserves current setting
>        :return: self

Documentation for `save_cmb_power_spectrum()`:
>        Save CMB power to a plain text file. Output is lensed total :math:`\ell(\ell+1)C_\ell/2\pi` then
>        lensing potential and cross: L TT EE BB TE PP PT PE.
>
>        :param filename: filename to save
>        :param lmax: lmax to save
>        :param CMB_unit: scale results from dimensionless. Use 'muK' for :math:`\mu K^2` units for CMB :math:`C_\ell`
>               and :math:`\mu K` units for lensing cross.

Documentation for `get_cmb_power_spectrum()`:
>        Get CMB power spectra, as requested by the 'spectra' argument. All power spectra are
>        :math:`\ell(\ell+1)C_\ell/2\pi` self-owned numpy arrays (0..lmax, 0..3), where 0..3 index
>        are TT, EE, BB, TE, unless raw_cl is True in which case return just :math:`C_\ell`.
>        For the lens_potential the power spectrum returned is that of the deflection.
>
>        Note that even if lmax is None, all spectra a returned to the same lmax, appropriate
>        for lensed spectra. Use the individual functions instead if you want to the full unlensed
>        and lensing potential power spectra to the higher lmax actually computed.
>
>        :param params: optional :class:`~.model.CAMBparams` instance with parameters to use. If None, must have
>          previously set parameters and called `calc_power_spectra` (e.g. if you got this instance
>          using :func:`.camb.get_results`),
>        :param lmax: maximum L
>        :param spectra: list of names of spectra to get
>        :param CMB_unit: scale results from dimensionless. Use 'muK' for :math:`\mu K^2` units for CMB :math:`C_\ell`
>          and :math:`\mu K` units for lensing cross.
>        :param raw_cl: return :math:`C_\ell` rather than :math:`\ell(\ell+1)C_\ell/2\pi`
>        :return: dictionary of power spectrum arrays, indexed by names of requested spectra

## Suggestion

Documentation for `save_cmb_power_spectrum()`:
>        Save CMB power to a plain text file. Output is lensed total :math:`\ell(\ell+1)C_\ell/2\pi` then
>        lensing potential and cross: L TT EE BB TE PP PT PE. If lmax is None, the default lmax may differ from what
>         was set by :meth:`.results.CAMBparams.set_for_lmax`
>
>        :param filename: filename to save
>        :param lmax: lmax to save
>        :param CMB_unit: scale results from dimensionless. Use 'muK' for :math:`\mu K^2` units for CMB :math:`C_\ell`
>               and :math:`\mu K` units for lensing cross.

Documentation for `get_cmb_power_spectrum()`:
>        Get CMB power spectra, as requested by the 'spectra' argument. All power spectra are
>        :math:`\ell(\ell+1)C_\ell/2\pi` self-owned numpy arrays (0..lmax, 0..3), where 0..3 index
>        are TT, EE, BB, TE, unless raw_cl is True in which case return just :math:`C_\ell`.
>        For the lens_potential the power spectrum returned is that of the deflection.
>
>        Note that even if lmax is None, all spectra a returned to the same lmax, appropriate
>        for lensed spectra (which may differ from an lmax specified in by :meth:`.results.CAMBparams.set_for_lmax`). 
>        Use the individual functions instead if you want to get the full unlensed
>        and lensing potential power spectra to the higher lmax actually computed.
>
>        :param params: optional :class:`~.model.CAMBparams` instance with parameters to use. If None, must have
>          previously set parameters and called `calc_power_spectra` (e.g. if you got this instance
>          using :func:`.camb.get_results`),
>        :param lmax: maximum L
>        :param spectra: list of names of spectra to get
>        :param CMB_unit: scale results from dimensionless. Use 'muK' for :math:`\mu K^2` units for CMB :math:`C_\ell`
>          and :math:`\mu K` units for lensing cross.
>        :param raw_cl: return :math:`C_\ell` rather than :math:`\ell(\ell+1)C_\ell/2\pi`
>        :return: dictionary of power spectrum arrays, indexed by names of requested spectra